In [2]:
# %%
"""
Claim Severity Modeling Pipeline
===============================

This notebook implements the complete pipeline for predicting claim severity
(TotalClaims for policies with claims > 0).

Business Objective:
- Predict the financial impact of claims to help with reserving and pricing
- Identify key factors that drive claim severity
- Support risk-based pricing decisions
"""

# %%
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sys
import os

# Add src to Python path
sys.path.append('../src')

from modeling.data_preprocessor import DataPreprocessor
from modeling.feature_engineer import FeatureEngineer
from modeling.model_trainer import ModelTrainer
from modeling.model_evaluator import ModelEvaluator
from modeling.model_interpreter import ModelInterpreter

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')

# %%
# Load the data
print("Loading insurance data...")
data = pd.read_csv('../data/raw/insurance_data.csv')  # Adjust path as needed
print(f"Data shape: {data.shape}")
print(f"Columns: {list(data.columns)}")

# %%
# Initialize preprocessing pipeline
preprocessor = DataPreprocessor(random_state=42)
feature_engineer = FeatureEngineer()

print("Preparing claim severity dataset...")
# Prepare data for claim severity modeling (only policies with claims)
claim_data = preprocessor.prepare_claim_severity_data(data)
print(f"Claim severity dataset shape: {claim_data.shape}")

# %%
# Feature Engineering
print("Engineering features...")

claim_data = feature_engineer.create_risk_features(claim_data)
claim_data = feature_engineer.create_interaction_features(claim_data)

print(f"Data shape after feature engineering: {claim_data.shape}")

# %%
# Data preprocessing
print("Encoding categorical features...")
claim_data_encoded = preprocessor.encode_categorical_features(claim_data, encoding_strategy='mixed')

# Prepare features and target
# Remove target and other non-feature columns
exclude_columns = ['TotalClaims', 'PolicyID', 'UnderwrittenCoverID', 'TransactionMonth']
feature_columns = [col for col in claim_data_encoded.columns if col not in exclude_columns]

X = claim_data_encoded[feature_columns]
y = claim_data_encoded['TotalClaims']

print(f"Feature matrix shape: {X.shape}")
print(f"Target variable shape: {y.shape}")

# %%
# Train-test split
print("Splitting data into train and test sets...")
X_train, X_test, y_train, y_test = preprocessor.split_data(X, y, test_size=0.2)

# Scale features
X_train_scaled, X_test_scaled = preprocessor.scale_features(X_train, X_test)

print(f"Training set shape: {X_train_scaled.shape}")
print(f"Test set shape: {X_test_scaled.shape}")

# %%
# Feature Selection
print("Performing feature selection...")
X_train_selected, selected_features = feature_engineer.select_features(
    X_train_scaled, y_train, method='mutual_info', k=20, problem_type='regression'
)

# Apply same feature selection to test set
X_test_selected = X_test_scaled[selected_features]

print(f"Selected features: {selected_features}")
print(f"Reduced feature matrix shape: {X_train_selected.shape}")

# %%
# Model Training
print("Training multiple models for claim severity prediction...")

model_trainer = ModelTrainer(random_state=42)

# Train all models (without hyperparameter tuning for speed, but you can enable it)
models = model_trainer.train_all_models(
    X_train_selected, y_train, 
    problem_type='regression', 
    tune_hyperparameters=False,  # Set to True for better performance
    cv_folds=5
)

print("Model training completed!")

# %%
# Model Evaluation
print("Evaluating models...")

evaluator = ModelEvaluator()

evaluation_results = {}

for model_name, model in models.items():
    print(f"\nEvaluating {model_name}...")
    eval_result = evaluator.evaluate_regression_model(
        model, model_name, X_test_selected, y_test, y_train, X_train_selected
    )
    evaluation_results[model_name] = eval_result

# %%
# Model Comparison
print("\nModel Comparison Results:")
comparison_df = evaluator.compare_models(metric='rmse', problem_type='regression')
print(comparison_df)

# Also compare R² scores
r2_comparison = evaluator.compare_models(metric='r2_score', problem_type='regression')
print("\nR² Score Comparison:")
print(r2_comparison)

# %%
# Visualize model performance
best_model_name = comparison_df.iloc[0]['model']
best_model = models[best_model_name]

print(f"\nCreating evaluation plots for best model: {best_model_name}")
evaluator.plot_regression_results(
    best_model, best_model_name, X_test_selected, y_test, 
    save_path='../reports/figures/modeling'
)

# %%
# Model Interpretation
print("Performing model interpretation...")

interpreter = ModelInterpreter()

# Feature importance analysis
feature_importance = interpreter.analyze_feature_importance(
    best_model, best_model_name, X_test_selected, y_test, 
    feature_names=selected_features, top_n=15
)

# %%
# SHAP Analysis
print("Computing SHAP values...")
shap_values, shap_importance = interpreter.analyze_shap_values(
    best_model, best_model_name, X_train_selected, X_test_selected, 
    feature_names=selected_features, sample_size=100
)

# %%
# Business Insights
print("Generating business insights...")
business_insights = interpreter.generate_business_insights(
    best_model_name, feature_names=selected_features
)

print("\nKey Risk Drivers:")
for driver in business_insights['key_risk_drivers'][:5]:
    print(f"- {driver['feature']}: {driver['business_meaning']}")

print("\nBusiness Recommendations:")
for rec in business_insights['business_recommendations']:
    print(f"- {rec}")

# %%
# Save Models and Results
print("Saving models and results...")

# Create directories if they don't exist
os.makedirs('../models/claim_severity', exist_ok=True)
os.makedirs('../reports/figures/modeling', exist_ok=True)

# Save models
model_trainer.save_models('../models/claim_severity/claim_severity_model')

# Save preprocessing objects
preprocessor.save_preprocessors('../models/claim_severity/preprocessor.joblib')

# Generate evaluation report
evaluation_report = evaluator.generate_evaluation_report(
    save_path='../reports/figures/modeling'
)

# Generate interpretation report
interpretation_report = interpreter.create_interpretation_report(
    best_model_name, save_path='../reports/figures/modeling'
)

print("Claim severity modeling pipeline completed successfully!")

# %%
# Final Results Summary
print("\n" + "="*50)
print("CLAIM SEVERITY MODELING SUMMARY")
print("="*50)

print(f"\nBest Model: {best_model_name}")
print(f"RMSE: {evaluation_results[best_model_name]['rmse']:.2f}")
print(f"R² Score: {evaluation_results[best_model_name]['r2_score']:.4f}")
print(f"MAPE: {evaluation_results[best_model_name]['mape']:.2f}%")

print(f"\nTop 5 Most Important Features:")
if best_model_name in interpreter.feature_importance:
    importance_data = interpreter.feature_importance[best_model_name]['built_in']
    top_features = sorted(importance_data.items(), key=lambda x: x[1], reverse=True)[:5]
    for i, (feature, importance) in enumerate(top_features, 1):
        print(f"{i}. {feature}: {importance:.4f}")

print("\nModel files saved to: ../models/claim_severity/")
print("Evaluation reports saved to: ../reports/figures/modeling/")

Loading insurance data...


INFO:modeling.data_preprocessor:Claim severity dataset: 2788 records with claims


Data shape: (1000098, 52)
Columns: ['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth', 'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province', 'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode', 'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders', 'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors', 'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser', 'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt', 'Converted', 'CrossBorder', 'NumberOfVehiclesInFleet', 'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm', 'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium', 'TotalClaims']
Preparing claim severity dataset...


INFO:modeling.data_preprocessor:Missing values handled. Remaining missing: 0


Claim severity dataset shape: (2788, 54)
Engineering features...


NameError: name 'bin_edgess' is not defined